# TASEPy applications: estimating the ribosome initiation rate $\alpha$ from experimental data

The totally asymmetric simple exclusion process (TASEP) has been introduced as a model of biopolymerization, where the mRNA is represented as a one-dimensional lattice with discrete sites corresponding to codons. The dynamics of ribosomes on the mRNA is modeled with particles moving from site to site on the lattice.

As an application of the TASEPy package, we demonstrate how to infer the initiation rate $\alpha$ when the hopping rates $\omega_i$ (representing codon-dependent ribosome speed) and the mean density $\rho$ (indicating the average number of ribosomes divided by the length of the mRNA in codon units) are known. We illustrate this in the context of mRNA translation, where the mean ribosome density has been measured using polysome profiling, and the codon-dependent hopping rates have been estimated based on tRNA concentrations.

Thus, by predicring $\rho(\alpha)$ with TASEPy and with the experimental value $\rho_\textrm{exp}$, we can infer the value of $\alpha$.

In [1]:
# import methods from TASEPy

from TASEPy import psa_compute
from TASEPy import local_density
from TASEPy import mean_density
from TASEPy import current

# to measure computation time
import time 

# root finding algorithm
from scipy import optimize

First, we import a file containing hopping rates for the YAL008W gene of *S. cerevisiae*, which have been estimated in Ciandrini L, Stansfield I, Romano MC (2013) *Ribosome Traffic on mRNAs Maps to Gene Ontology: Genome-wide Quantification of Translation Initiation Rates and Polysome Size Regulation*. PLoS Comput Biol 9(1): e1002866. https://doi.org/10.1371/journal.pcbi.1002866

In [2]:
# imports hopping rates YAL008W_rates.dat

with open('applications/YAL008W_rates.dat', 'r') as file:
    rates = [float(line.split()[1]) for line in file]

Next, we set particle size $\ell=9$ as in the reference above, and choose the order of PSA $K=3$. This takes about 11 seconds to solve on a laptop Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz with 16 GB of RAM, less than 7 seconds on an Apple M1 Pro with 32 GB of RAM.

In [3]:
# computes the PSA for order K = 3

L = len(rates) # lattice size
ll = 9 # particle size
K = 3 # maximum PSA order

print('Lattice size:',L,'codons')

start = time.time()
rhocoeff, Jcoeff = psa_compute(rates, K, ll)
end = time.time()
print('Computation time:',round(end-start,3),'seconds')

Lattice size: 198 codons
Computation time: 6.775 seconds


We then need to compute the mean density coefficients $\rho_n$ from all the $\rho_{i,n}$ as in Eq.(25c) of the paper introducing the algorothm used in TASEPy (Ciandrini, Crisostomo and Szavits-Nossan 2023).

In [4]:
# computes mean density coefficients

mean_rhocoeff = []
for order in range(0,K+1):
    rhocoeff_sum = 0
    for site in range(L):
        rhocoeff_sum += rhocoeff[site][order]
    mean_rhocoeff.append(rhocoeff_sum/L)
print(mean_rhocoeff)

[0.0, 0.1806069864721105, -0.20947619500711065, 0.32647921092495436]


Next, we set up the function whose root we want to find: $f(\alpha)=\rho(\alpha)-\rho_{\text{exp}}$, where $\rho(\alpha)$ is the mean density computed using TASEPy for a given value of the initiation rate $\alpha$, and $\rho_{\text{exp}}$ is the mean density measured experimentally, see the cited Ciandrini et (2013) reference for more details. Since $f(\alpha)$ is a polynomial, we can also find its derivative, in which case we can use the Newton-Raphson method for solving $f(\alpha)=0$.

In [5]:
# experimental mean density
rhoexp = 0.023226

# in the functions below x is the initation rate (alpha)
def f(x,a):
    result = -rhoexp
    for order,coeff in enumerate(a):
        result += coeff * x**order
    return result

def fprime(x,a):
    result = 0
    for order,coeff in enumerate(a):
        if order > 0:
            result += coeff * order * x**(order-1)
    return result

As the initial guess, we can use the first-order approximation of the PSA.

In [6]:
# initial guess
alpha0 = rhoexp/mean_rhocoeff[1]
print('Initial value:',alpha0)

Initial value: 0.12859967631200458


Finally, we call the optimize function from the scipy package to find the root.

In [7]:
alpha = optimize.newton(f, alpha0, fprime, args=(mean_rhocoeff,))
print('Optimal value:',alpha)

Optimal value: 0.14818676243771686


This value is very close to the one obtained from stochastic simulations in the cited Ciandrini et al (2013) paper. The relative error is about 1.5\%.

In [8]:
# value reported in Ciandrini L, Stansfield I, Romano MC (2013) 
# Ribosome Traffic on mRNAs Maps to Gene Ontology: Genome-wide 
# Quantification of Translation Initiation Rates and Polysome 
# Size Regulation. PLoS Comput Biol 9(1): e1002866. 
# https://doi.org/10.1371/journal.pcbi.1002866

alpha_paper = 0.150499
per_err = 100 * abs(alpha_paper-alpha)/alpha_paper
print('Relative percentage error: ',per_err)

Relative percentage error:  1.5363806817873418
